In [1]:
pip install bcrypt requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 1.5 MB/s eta 0:00:00


In [2]:
import csv
import bcrypt
import re
import requests
from datetime import datetime, timedelta

# Constants
CSV_FILE = 'regno.csv'
API_URL = 'https://api.sunrise-sunset.org/json'
GEOCODE_API_URL = 'https://api.opencagedata.com/geocode/v1/json'  # Dummy geocoding API
GEOCODE_API_KEY = 'YOUR_API_KEY'  # Replace with your OpenCage API key

# Function to validate email
def is_valid_email(email):
    regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(regex, email)

# Function to validate password
def is_valid_password(password):
    return (len(password) >= 8 and
            any(c.isupper() for c in password) and
            any(c.islower() for c in password) and
            any(c.isdigit() for c in password) and
            any(c in '!@#$%^&*()-=_+[]{}|;:,.<>?' for c in password))

# Function to hash password
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

# Function to check password
def check_password(hashed_password, password):
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password)

# Function to read user data from CSV
def read_users():
    users = {}
    try:
        with open(CSV_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                users[row['email']] = {
                    'hashed_password': row['hashed_password'].encode('utf-8'),
                    'security_question': row['security_question'],
                }
    except FileNotFoundError:
        pass
    return users

# Function to write user data to CSV
def write_users(users):
    with open(CSV_FILE, mode='w', newline='') as file:
        fieldnames = ['email', 'hashed_password', 'security_question']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for email, data in users.items():
            writer.writerow({
                'email': email,
                'hashed_password': data['hashed_password'].decode('utf-8'),
                'security_question': data['security_question'],
            })

# Function for user login
def login(users):
    attempts = 5
    while attempts > 0:
        email = input("Enter your email: ")
        password = input("Enter your password: ")

        if email in users and check_password(users[email]['hashed_password'], password):
            print("Login successful!")
            return email
        else:
            attempts -= 1
            print(f"Invalid credentials. {attempts} attempts remaining.")

    print("Too many failed attempts. Exiting...")
    exit()

# Function for password recovery
def password_recovery(users):
    email = input("Enter your registered email: ")
    if email in users:
        print(f"Security Question: {users[email]['security_question']}")
        answer = input("Answer the security question: ")
        # In a real application, you would verify the answer here.
        new_password = input("Enter your new password: ")
        if is_valid_password(new_password):
            users[email]['hashed_password'] = hash_password(new_password)
            write_users(users)
            print("Password reset successful.")
        else:
            print("Invalid password format.")
    else:
        print("Email not found.")

# Function to get sunset and sunrise times
def get_sun_times(location):
    response = requests.get(API_URL, params={'lat': location['lat'], 'lng': location['lng'], 'formatted': 0})
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            results = data['results']
            print(f"Sunrise: {results['sunrise']}")
            print(f"Sunset: {results['sunset']}")
            print(f"Day Length: {results['day_length']} seconds")
            print(f"Solar Noon: {results['solar_noon']}")
        else:
            print("Invalid location. Please verify and try again.")
    else:
        print("Error fetching data from API. Please check your internet connection.")

# Function to get location coordinates
def get_location_coordinates(city):
    response = requests.get(GEOCODE_API_URL, params={'q': city, 'key': GEOCODE_API_KEY})
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            lat = data['results'][0]['geometry']['lat']
            lng = data['results'][0]['geometry']['lng']
            return {'lat': lat, 'lng': lng}
        else:
            print("Location not found.")
            return None
    else:
        print("Error fetching location data.")
        return None

# Main application logic
def main():
    users = read_users()

    while True:
        choice = input("1. Login\n2. Forgot Password\n3. Exit\nChoose an option: ")
        if choice == '1':
            email = login(users)
            city = input("Enter the city for sunset and sunrise times: ")
            location = get_location_coordinates(city)
            if location:
                get_sun_times(location)
        elif choice == '2':
            password_recovery(users)
        elif choice == '3':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


1. Login
2. Forgot Password
3. Exit
Choose an option: 1
Enter your email: vijethabadveli03@gmail.com
Enter your password: jasmitha@03
Invalid credentials. 4 attempts remaining.
Enter your email: vijethabadveli03@gmail.com
Enter your password: jasmitha@03
Invalid credentials. 3 attempts remaining.
Enter your email: vijethabadveli03@gmail.com
Enter your password: Jasmitha@03
Invalid credentials. 2 attempts remaining.
Enter your email: vijethabadveli03@gmail,com
Enter your password: Jasmitha@03
Invalid credentials. 1 attempts remaining.
Enter your email: vijethabadveli03@gmail.com
Enter your password: Jasmitha@03
Invalid credentials. 0 attempts remaining.
Too many failed attempts. Exiting...
Enter the city for sunset and sunrise times: Andhra pradesh
Error fetching location data.
1. Login
2. Forgot Password
3. Exit
Choose an option: 2
Enter your registered email: vijethabadveli03@gmail.com
Email not found.
1. Login
2. Forgot Password
3. Exit
Choose an option: 3
Exiting...
